# Sorting Networks

## Daniel, Swati, and Michael

### Classes for Sorting Networks
Elements are objects that take in two inputs, A and B, and then output L and H with L = A, H = B if A < B or L = B, H = A if A > B. Each element instance tracks whether the two inputs were swapped. 

Layers are arrays of elements

In [1]:
import random

class Element:
    
    def swap(self, A, B, descending):   
        if A <= B:
            self.swapped = False
            self.L = A
            self.H = B
        else:
            self.swapped = True
            self.L = B
            self.H = A
        
        if descending:
            temp = self.L
            self.L = self.H
            self.H = temp
            self.swapped = not self.swapped
            
    def __init__(self, A, B, descending = False):
        self.prob = 1
        self.A = A
        self.B = B
        self.descending = descending
        self.swap(A, B, descending)
        
class Layer:
    def __init__(self, elements, indices):
        self.in_indices = indices
        self.elements = elements
        self.next = None
        self.out_indices = []
        for i, element in enumerate(elements):
            if element.swapped:
                self.out_indices.append(self.in_indices[i * 2 + 1])
                self.out_indices.append(self.in_indices[i * 2])
            else:
                self.out_indices.append(self.in_indices[i * 2])
                self.out_indices.append(self.in_indices[i * 2 + 1])
    
    def get_numbers(self):
        numbers = []
        for element in self.elements:
            numbers.append(element.L)
            numbers.append(element.H)
        return numbers
    
    def get_inputs(self):
        numbers = []
        for element in self.elements:
            numbers.append(element.A)
            numbers.append(element.B)
        return numbers
    
    def print_layer(self):
        print(self.get_numbers())
        
    def get_end_layer(self):
        current_layer = self
        while current_layer.next is not None:
            current_layer = current_layer.next
        return current_layer
    
    def get_end(self):
        return self.get_end_layer().get_numbers()
    
    def print_end(self):
        print(self.get_end())
    
    def get_depth(self):
        depth = 1
        current_layer = self
        while current_layer.next is not None:
            current_layer = current_layer.next
            depth = depth + 1
        return depth
    
    def copy(self, layer):
        self.elements = layer.elements
        self.next = layer.next
        self.in_indices = layer.in_indices
        self.out_indices = layer.out_indices
        
    def count_swaps(self):
        swaps = 0
        current_layer = self
        while current_layer is not None:
            for element in current_layer.elements:
                if element.swapped:
                    swaps = swaps + 1
            current_layer = current_layer.next
        return swaps
    
    def to_list(self):
        layer_list = []
        current_layer = self
        while current_layer is not None:
            layer_list.append(current_layer.elements)
            current_layer = current_layer.next
        return layer_list

def combine_sublayers(sublayers):
    if len(sublayers) is 0:
        return None
    else:
        total_elements = []
        next_sublayers = []
        indices = []
        for sublayer in sublayers:
            if sublayer.next is not None:
                next_sublayers.append(sublayer.next)
            total_elements = total_elements + sublayer.elements
            indices = indices + sublayer.in_indices
        current_layer = Layer(total_elements, indices)
        current_layer.next = combine_sublayers(next_sublayers)

        return current_layer


In [2]:
def bitonic_sorter(a, indices, reverse = False):
    # print("SORTING {} WITH INDICES {}".format(a, indices))
    if len(a) == 2:
        # If there are only two elements to sort, the layer is only one comparator
        layer = Layer([Element(a[0], a[1], reverse)], indices)
        return layer
    else:
        # print("CALLING BITONIC SORTER")
        # Else, we create layer with inputs (1, n/2 + 1), (2, n/2 + 2), ...
        bitonic_halver = []
        halver_indices = []
        for i in range(0, len(a) // 2):
            bitonic_halver.append(Element(a[i], a[i + len(a) // 2], reverse))
            halver_indices.append(indices[i])
            halver_indices.append(indices[i + len(a) // 2])
        layer = Layer(bitonic_halver, halver_indices)
        low_numbers = []
        high_numbers = []
        out_indices = layer.out_indices
        # All low elements of the last layer get passed into another bitonic sorter
        # All high elements of the last layer get passed into another bitonic sorter 
        low_indices = []
        high_indices = []
        for i in range(0, len(layer.elements)):
            low_numbers.append(layer.elements[i].L)
            high_numbers.append(layer.elements[i].H)
            low_indices.append(out_indices[2 * i])
            high_indices.append(out_indices[2 * i + 1])

        # print("Low indices: {}".format(low_indices))
        # print("High indices: {}".format(high_indices))
        # print()
        lower_layer = bitonic_sorter(low_numbers, low_indices, reverse)
        higher_layer = bitonic_sorter(high_numbers, high_indices, reverse)
        
        # We combine the outputs of the low layer
        layer.next = combine_sublayers([lower_layer, higher_layer])
        indices = layer.out_indices
        return layer
    
    
def bitonic_converter(a):
    sort_size = 2
    top_layer = Layer([], [])
    current_layer = top_layer
    current_indices = list(range(0, len(a)))
    while sort_size < len(a):
        # Sort size is 2, 4, 8, ... 2^(n - 1)
        current_layer.next = Layer([], current_indices)
        current_layer = current_layer.next
        sublayers = []
        sublayer_out_indices = []
        reverse = False
        # Sort every sort_size elements in alternating up down sequences
        for i in range(0, len(a) // sort_size):
            sublayer_indices = current_indices[(i) * sort_size:(i + 1) * sort_size]
            sublayers.append(bitonic_sorter(a[(i) * sort_size:(i + 1) * sort_size], sublayer_indices, reverse))
            sublayer_out_indices = sublayer_out_indices + sublayers[i].get_end_layer().out_indices
            reverse = not reverse
        # Combine the sublayers and pass output into the next sorting layer
        current_layer.copy(combine_sublayers(sublayers))
        current_layer = current_layer.get_end_layer()
        current_indices = sublayer_out_indices
        # current_layer.out_indices = sublayer_out_indices
        a = current_layer.get_numbers()
        sort_size = sort_size * 2
    return top_layer.next

def sort(a):
    bitonic_layers = bitonic_converter(a)
    b = bitonic_layers.get_end()
    bitonic_indices = bitonic_layers.get_end_layer().out_indices
    sorting_layers = bitonic_sorter(b, bitonic_indices)
    bitonic_layers.get_end_layer().next = sorting_layers
    return bitonic_layers

In [3]:
p = 4
a = [59, 74, 54, 69, 42, 99, 0, 52, 2, 98, 87, 84, 4, 82, 90, 1]
print(a)
bitonic_layers = bitonic_converter(a)
bitonic_out_indices = bitonic_layers.get_end_layer().out_indices
b = bitonic_layers.get_end()
print(b)
print([a[i] for i in bitonic_out_indices])
print()

[59, 74, 54, 69, 42, 99, 0, 52, 2, 98, 87, 84, 4, 82, 90, 1]
[0, 42, 52, 54, 59, 69, 74, 99, 98, 90, 87, 84, 82, 4, 2, 1]
[0, 42, 52, 54, 59, 69, 74, 99, 98, 90, 87, 84, 82, 4, 2, 1]



In [4]:
p = 4
print("p = {}, array size = {}".format(p, 2**p))
a = random.sample(range(0, 100), 2**p)
print("ORIGINAL ARRAY: {}".format(a))
sorting_network = sort(a)
print("SORTED ARRAY: {}".format(sorting_network.get_end()))
print("SORT MAPPING: {}".format(sorting_network.get_end_layer().out_indices))
print([a[i] for i in sorting_network.get_end_layer().out_indices])

p = 4, array size = 16
ORIGINAL ARRAY: [92, 96, 49, 8, 51, 17, 34, 47, 28, 73, 14, 58, 33, 54, 85, 2]
SORTED ARRAY: [2, 8, 14, 17, 28, 33, 34, 47, 49, 51, 54, 58, 73, 85, 92, 96]
SORT MAPPING: [15, 3, 10, 5, 8, 12, 6, 7, 2, 4, 13, 11, 9, 14, 0, 1]
[2, 8, 14, 17, 28, 33, 34, 47, 49, 51, 54, 58, 73, 85, 92, 96]


### Perturbations Test

In [5]:
# Applies the mapping from a sorting network onto a perturbed array and counts the mistakes
def run_perturbation_test_1(p, e, output = False):

    a = random.sample(range(0, 10), 2**p)
    perturbations = [round(random.uniform(-e, e), 3) for num in range(0, len(a))]
    perturbed_a = [a[i] + perturbations[i] for i in range(0, len(a))]

    sorting_network = sort(a)
    sorted_a = sorting_network.get_end()
    sorted_mapping = sorting_network.get_end_layer().out_indices

    sorted_perturbed_a = [perturbed_a[i] for i in sorted_mapping]
    
    true_sorted_perturbed_a = sorted(sorted_perturbed_a)
    
    mistakes = sum([0 if true_sorted_perturbed_a[i] == sorted_perturbed_a[i] else 1 for i in range(0, len(a))])
    
    if output:
        print("p = {}, array size = {}".format(p, 2**p))
        print("ORIGINAL ARRAY: {}".format(a))
        print("SORTED ARRAY: {}".format(sorted_a))
        print("PERTURBED ARRAY: {}".format(perturbed_a))
        print("SORTED MAPPING: {}".format(sorted_mapping))
        print("SORT MAPPING APPLIED ON PERTURBED ARRAY: {}".format(sorted_perturbed_a))
        print("TRUE SORTED PERTURBED ARRAY: {}".format(true_sorted_perturbed_a))
    
    return mistakes

# Applies an existing sorting network on array a, using only elements that had swaps
def apply_sorting_network(a, sorting_network):
    layer_input = a
    current_layer = sorting_network
    while current_layer is not None:
        layer_output = []
        for i, element in enumerate(current_layer.elements):
            # We only apply swapping if the element was used in the sorting network
            A = layer_input[2 * i]
            B = layer_input[2 * i + 1]
            if element.swapped:
                element_clone = Element(A, B, descending = element.descending)
                layer_output.append(element_clone.L)
                layer_output.append(element_clone.H)
            else:
                layer_output.append(A)
                layer_output.append(B)
        current_layer_out_indices = current_layer.out_indices
        if current_layer.next is None:
            break
        next_layer_in_indices = current_layer.next.in_indices
        out_index_map = {current_layer_out_indices[i]: layer_output[i] for i in range(0, len(a))}
        layer_input = [out_index_map[i] for i in next_layer_in_indices]
        current_layer = current_layer.next
    return layer_output

def run_perturbation_test_2(p, e, output = False):
    a = list(range(0, 2**p))
    random.shuffle(a)
    print(a)
    perturbations = [round(random.uniform(-e, e), 3) for num in range(0, len(a))]
    perturbed_a = [round(a[i] + perturbations[i], 3) for i in range(0, len(a))]

    sorting_network = sort(a)
    sorted_perturbed_a = apply_sorting_network(perturbed_a, sorting_network)
    
    sorted_a = sorting_network.get_end()
    true_sorted_perturbed_a = sorted(sorted_perturbed_a)
    
    mistakes = sum([0 if true_sorted_perturbed_a[i] == sorted_perturbed_a[i] else 1 for i in range(0, len(a))])
    
    if output:
        print("p = {}, array size = {}".format(p, 2**p))
        print("ORIGINAL ARRAY: {}".format(a))
        print("SORTED ARRAY: {}".format(sorted_a))
        print("PERTURBED ARRAY: {}".format(perturbed_a))
        print("PERTURBED ARRAY AFTER SORTING NETWORK: {}".format(sorted_perturbed_a))
        print("TRUE SORTED PERTURBED ARRAY: {}".format(true_sorted_perturbed_a))
        
    return sorting_network, mistakes

In [6]:
## Probablistic Sorting Test
print("--------- RUNNING PROBABLISTIC SORTING TEST 1 ---------")
print()

# Generate base array 

p = 10
a = list(range(0, 2**p))
random.shuffle(a)
true_sorted = list(range(0, 2**p))
# print("BASE ARRAY: {}".format(a))
# print("SORTED ARRAY: {}".format(true_sorted))

# Get mapping
sort_map = {}
for i, num in enumerate(a):
    sort_map[i] = true_sorted.index(num)
    
# print("SORTED MAPPING: {}".format(sort_map))

# print()

e = 1
# print("PERTURBATION BOUND: {}".format(e))

perturbed_sort_maps = []

# Over a set amount of iterations, we sort a perturbed vector through the sorting network

for perturbation in range(0, 1000):
    # Create the perturbed array
    perturbations = [round(random.uniform(-e, e), 4) for i in range(0, len(a))]
    perturbed_a = [round(a[i] + perturbations[i], 4) for i in range(0, len(a))]
    # print("PERTURBED ARRAY: {}".format(perturbed_a))

    # Get sorting network for original array
    sorting_network = sort(a)

    # Apply sorting network and get mapping
    sorted_perturbed_a = apply_sorting_network(perturbed_a, sorting_network)
    # print("SORTED PERTURBED ARRAY: {}".format(sorted_perturbed_a))

    true_sorted_perturbed_a = sorted(sorted_perturbed_a)

    mistakes = sum([0 if true_sorted_perturbed_a[i] == sorted_perturbed_a[i] else 1 for i in range(0, len(a))])

    # Get mapping
    perturbed_sort_map = {}
    for i, num in enumerate(perturbed_a):
        perturbed_sort_map[i] = sorted_perturbed_a.index(num)

    perturbed_sort_maps.append(perturbed_sort_map)

    # print("PERTURBED SORTED MAPPING: {}".format(perturbed_sort_map))
    # print("MISTAKES: {}".format(mistakes))
    # print()

prob_sort_map = {}
for i in range(0, 2**p):
    # print("WORKING ON INDEX {}".format(i))
    out_map = {}
    total_outs = 0
    for perturbed_sort_map in perturbed_sort_maps:
        out_index = perturbed_sort_map[i]
        if out_index not in out_map:
            out_map[out_index] = 0
        out_map[out_index] = out_map[out_index] + 1
        total_outs = total_outs + 1
        
    prob_sort_map[i] = {}
    for out in out_map:
        prob_sort_map[i][out] =  out_map[out] / total_outs
    
# print(prob_sort_map)
# print()
def apply_prob_sort_map(prob_sort_map, a):
    sorted_array = [-1 for i in range(0, len(a))]
    for i in range(0, len(a)):
        out_map = prob_sort_map[i]
        # print("OUT MAP: {}".format(out_map))
        num = a[i]
        # print("NUM: {}".format(num))
        random_num = random.uniform(0, 1)
        # print("RANDOM NUM: {}".format(random_num))
        for out in out_map:
            if random_num <= out_map[out]:
                sorted_array[out] = num
                # print("OUT: {}".format(out))
                break
            else:
                random_num = random_num - out_map[out]
        # print()
    return sorted_array

apply_prob_sort_map(prob_sort_map, a)

all_mistakes = []
for i in range(0, 1000):
    perturbations = [round(random.uniform(-e, e), 4) for i in range(0, len(a))]
    perturbed_a = [round(a[i] + perturbations[i], 4) for i in range(0, len(a))]


    sorted_perturbed_a = apply_prob_sort_map(prob_sort_map, perturbed_a)

    true_sorted_perturbed_a = sorted(sorted_perturbed_a)

    mistakes = sum([0 if true_sorted_perturbed_a[i] == sorted_perturbed_a[i] else 1 for i in range(0, len(a))])
    all_mistakes.append(mistakes)
    # print(mistakes)
print("AVERAGE MISTAKES: {}".format(sum(all_mistakes) / len(all_mistakes)))

--------- RUNNING PROBABLISTIC SORTING TEST 1 ---------

AVERAGE MISTAKES: 1000.75


In [7]:
### Probablistic Sorting Test 2
print("--------- RUNNING PROBABLISTIC SORTING TEST 2 ---------")
# print()

# Generate base array 

p = 10
a = list(range(0, 2**p))
random.shuffle(a)
true_sorted = list(range(0, 2**p))
# print("BASE ARRAY: {}".format(a))
# print("SORTED ARRAY: {}".format(true_sorted))

# Get mapping
sort_map = {}
for i, num in enumerate(a):
    sort_map[true_sorted.index(num)] = i
    
# print("SORTED MAPPING: {}".format(sort_map))

# print()

e = 1
# print("PERTURBATION BOUND: {}".format(e))

perturbed_sort_maps = []

# Over a set amount of iterations, we sort a perturbed vector through the sorting network

for perturbation in range(0, 1000):
    # print("RUNNING SIM {}".format(perturbation))
    # Create the perturbed array
    perturbations = [round(random.uniform(-e, e), 4) for i in range(0, len(a))]
    perturbed_a = [round(a[i] + perturbations[i], 4) for i in range(0, len(a))]
    # print("PERTURBED ARRAY: {}".format(perturbed_a))

    # Get sorting network for original array
    sorting_network = sort(a)

    # Apply sorting network and get mapping
    sorted_perturbed_a = apply_sorting_network(perturbed_a, sorting_network)
    # print("SORTED PERTURBED ARRAY: {}".format(sorted_perturbed_a))

    true_sorted_perturbed_a = sorted(sorted_perturbed_a)

    mistakes = sum([0 if true_sorted_perturbed_a[i] == sorted_perturbed_a[i] else 1 for i in range(0, len(a))])

    # Get mapping
    perturbed_sort_map = {}
    for i, num in enumerate(perturbed_a):
        perturbed_sort_map[sorted_perturbed_a.index(num)] = i
        # print("KEY {} GETS NUMBER {}".format(sorted_perturbed_a.index(num), i))

    perturbed_sort_maps.append(perturbed_sort_map)

    # print("PERTURBED SORTED MAPPING: {}".format(perturbed_sort_map))
    # print("MISTAKES: {}".format(mistakes))
    # print()

prob_sort_map = {}
for i in range(0, 2**p):
    in_map = {}
    total_ins = 0
    for j, perturbed_sort_map in enumerate(perturbed_sort_maps):
        try:
            in_index = perturbed_sort_map[i]
            if in_index not in in_map:
                in_map[in_index] = 0
            in_map[in_index] = in_map[in_index] + 1
            total_ins = total_ins + 1
        except:
            print("ERROR")
            # print(j)
            # print(perturbed_sort_map)
            # print(i)
            # print("HERE")
        
    prob_sort_map[i] = {}
    for num in in_map:
        prob_sort_map[i][num] =  in_map[num] / total_ins
        
# print(prob_sort_map)

def apply_prob_sort_map(prob_sort_map, a):
    sorted_array = [-1 for i in range(0, len(a))]
    for i in range(0, len(a)):
        in_map = prob_sort_map[i]
        # print("IN MAP: {}".format(in_map))
        random_num = random.uniform(0, 1)
        # print("RANDOM NUM: {}".format(random_num))
        for num in in_map:
            if random_num <= in_map[num]:
                # print("IN: {}".format(num))
                sorted_array[i] = a[num]
                break
            else:
                random_num = random_num - in_map[num]
        # print()
    return sorted_array
print()
all_mistakes = []
for i in range(0, 1000):
    perturbations = [round(random.uniform(-e, e), 4) for i in range(0, len(a))]
    perturbed_a = [round(a[i] + perturbations[i], 4) for i in range(0, len(a))]


    sorted_perturbed_a = apply_prob_sort_map(prob_sort_map, perturbed_a)

    true_sorted_perturbed_a = sorted(sorted_perturbed_a)

    mistakes = sum([0 if true_sorted_perturbed_a[i] == sorted_perturbed_a[i] else 1 for i in range(0, len(a))])
    all_mistakes.append(mistakes)
    # print(mistakes)
print("AVERAGE MISTAKES: {}".format(sum(all_mistakes) / len(all_mistakes)))

--------- RUNNING PROBABLISTIC SORTING TEST 2 ---------
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR
ERROR

AVERAGE MISTAKES: 210.04


In [8]:
### Probablistic Sorting Test 3
print("--------- RUNNING PROBABLISTIC SORTING TEST 3 ---------")
print()

# Generate base array 

p = 10
a = list(range(0, 2**p))
random.shuffle(a)
true_sorted = list(range(0, 2**p))
# print("BASE ARRAY: {}".format(a))
# print("SORTED ARRAY: {}".format(true_sorted))

# print()

e = 1
# print("PERTURBATION BOUND: {}".format(e))

perturbed_networks = []

prob_sorting_network = sort(a)

for i in range(0, 1000):
    # Create the perturbed array
    perturbations = [round(random.uniform(-e, e), 4) for i in range(0, len(a))]
    perturbed_a = [round(a[i] + perturbations[i], 4) for i in range(0, len(a))]
    # print("PERTURBED ARRAY: {}".format(perturbed_a))
    
    perturbed_network = sort(perturbed_a)
    perturbed_networks.append(perturbed_network)

network_probs = []
for i, layer in enumerate(prob_sorting_network.to_list()):
    layer_probs = []
    for perturbed_network in perturbed_networks:
        element_probs = [0 for i in range(0, len(perturbed_network.elements))]
        for j, element in enumerate(perturbed_network.to_list()[i]):
            if element.swapped:
                element_probs[j] = element_probs[j] + 1 / len(perturbed_networks)
        layer_probs.append(element_probs)
    layer_probs = np.array(layer_probs)
    layer_probs = np.sum(layer_probs, 0)
    network_probs.append(layer_probs)
    
layer_index = 0
current_layer = prob_sorting_network
while current_layer is not None:
    element_probs = network_probs[layer_index]
    for i, element in enumerate(current_layer.elements):
        element.prob = element_probs[i]
    current_layer = current_layer.next
    layer_index = layer_index + 1
    
# Applies an existing sorting network on array a, using only elements that had swaps
def apply_prob_sorting_network(a, sorting_network):
    layer_input = a
    current_layer = sorting_network
    while current_layer is not None:
        layer_output = []
        for i, element in enumerate(current_layer.elements):
            # We only apply swapping if the element was used in the sorting network
            A = layer_input[2 * i]
            B = layer_input[2 * i + 1]
            random_num = random.uniform(0, 1)
            if random_num <= element.prob:
                element_clone = Element(A, B, descending = element.descending)
                layer_output.append(element_clone.L)
                layer_output.append(element_clone.H)
            else:
                layer_output.append(A)
                layer_output.append(B)
        current_layer_out_indices = current_layer.out_indices
        if current_layer.next is None:
            break
        next_layer_in_indices = current_layer.next.in_indices
        out_index_map = {current_layer_out_indices[i]: layer_output[i] for i in range(0, len(a))}
        layer_input = [out_index_map[i] for i in next_layer_in_indices]
        current_layer = current_layer.next
    return layer_output
# print()

all_mistakes = []
for i in range(0, 1000):
    perturbations = [round(random.uniform(-e, e), 4) for i in range(0, len(a))]
    perturbed_a = [round(a[i] + perturbations[i], 4) for i in range(0, len(a))]


    sorted_perturbed_a = apply_prob_sorting_network(perturbed_a, prob_sorting_network)

    true_sorted_perturbed_a = sorted(sorted_perturbed_a)

    mistakes = sum([0 if true_sorted_perturbed_a[i] == sorted_perturbed_a[i] else 1 for i in range(0, len(a))])
    all_mistakes.append(mistakes)
    # print(mistakes)
    
print("AVERAGE MISTAKES: {}".format(sum(all_mistakes) / len(all_mistakes)))

--------- RUNNING PROBABLISTIC SORTING TEST 3 ---------



NameError: name 'np' is not defined